# Análisis de datos con Spark

In [0]:
# aunque en la guía decia cargarlo como volumen, lo cargamos como tablas. Solo de esta manera logramos importar los csv

vehiculos_involucrados = spark.table("default.vehiculos_involucrados_corregido")
fallecidos_lesionados = spark.table("default.fallecidos_lesionados_corregido")
hechos_transito = spark.table("default.hechos_transito_corregido")

#1 Mostrar cuantos registros hay en cada uno
print("vehiculos_involucrados: ", vehiculos_involucrados.count())
print("hechos_transito: ", hechos_transito.count())
print("fallecidos_lesionados: ", fallecidos_lesionados.count()) 

# y usar la funcion .show para ver algunos resultados de cada uno
vehiculos_involucrados.show(5)
print()
hechos_transito.show(5)
print()
fallecidos_lesionados.show(5)


vehiculos_involucrados:  111988
hechos_transito:  76759
fallecidos_lesionados:  109417
+---------+-------+-------+--------+---------+---------+---------+--------+--------+---------+----------+---------+---------+----------+--------+--------+--------------------+--------------------+----+
|num_hecho|dia_ocu|mes_ocu|hora_ocu|depto_ocu|mupio_ocu|areag_ocu|zona_ocu|tipo_veh|color_veh|modelo_veh|causa_acc|marca_veh|estado_pil|sexo_pil|edad_pil|      archivo_origen|        tipo_dataset| año|
+---------+-------+-------+--------+---------+---------+---------+--------+--------+---------+----------+---------+---------+----------+--------+--------+--------------------+--------------------+----+
|      1.0|    1.0|    1.0|     3.0|      1.0|    106.0|      1.0|     6.0|     3.0|     99.0|       6.0|      2.0|     99.0|       2.0|     1.0|    32.0|vehiculos_involuc...|vehiculos_involuc...|2013|
|      2.0|    1.0|    1.0|     5.0|      1.0|    101.0|      1.0|    11.0|     1.0|      2.0|       5.0|

In [0]:
# Hacer un summary para las columnas importantes de cada uno
# Versión compacta del summary
print("\nVARIABLES PRINCIPALES DE CADA ARCHIVO:")

print("\nVEHÍCULOS INVOLUCRADOS - Summary:")
vehiculos_summary = vehiculos_involucrados.select(
    "edad_pil", "mes_ocu", "hora_ocu"
).summary("count", "min", "max")
vehiculos_summary.show()

print("\nHECHOS DE TRÁNSITO - Summary:")
hechos_summary = hechos_transito.select(
    "mes_ocu", "hora_ocu", "dia_ocu", "dia_sem_ocu" 
).summary("count", "min", "max")
hechos_summary.show()

print("\nFALLECIDOS Y LESIONADOS - Summary:")
fallecidos_summary = fallecidos_lesionados.select(
    "edad_pil", "mes_ocu", "hora_ocu", "tipo_eve"
).summary("count", "min", "max")
fallecidos_summary.show()


VARIABLES PRINCIPALES DE CADA ARCHIVO:

VEHÍCULOS INVOLUCRADOS - Summary:
+-------+--------+-------+--------+
|summary|edad_pil|mes_ocu|hora_ocu|
+-------+--------+-------+--------+
|  count|  111988| 111988|  111988|
|    min|     3.0|    1.0|     0.0|
|    max|   999.0|   12.0|    99.0|
+-------+--------+-------+--------+


HECHOS DE TRÁNSITO - Summary:
+-------+-------+--------+-------+-----------+
|summary|mes_ocu|hora_ocu|dia_ocu|dia_sem_ocu|
+-------+-------+--------+-------+-----------+
|  count|  76759|   76759|  76759|      76759|
|    min|    1.0|     0.0|    1.0|        1.0|
|    max|   12.0|    99.0|   31.0|        7.0|
+-------+-------+--------+-------+-----------+


FALLECIDOS Y LESIONADOS - Summary:
+-------+--------+-------+--------+--------+
|summary|edad_pil|mes_ocu|hora_ocu|tipo_eve|
+-------+--------+-------+--------+--------+
|  count|  100427| 109417|  109417|  100357|
|    min|     0.0|    1.0|     0.0|     1.0|
|    max|   999.0|   12.0|    99.0|    99.0|
+----

Podemos ver que si se han cargado los datos de manera exitosa usando spark desde databricks, tuvimos que cargar los csv como tablas, pero aparte de eso no hemos hecho mayores cambios. Primero podemos ver algunas estadísticas usando summary donde podemos ver cosas como hora de ocurrencia, mes de ocurrencia entre otros. Algo interesante a mencionar con estos resultados, es que se puede ver que hay valores muy altos para la edad de los pilotos. Esto no necesariamente es un error del script, de hecho de acuerdo con el libro de variables que se puede encontrar en la página del INE, esto hace sentido. Las personas que ingresan los datos tienden a colocar valores muy altos en casillas numéricas cuando estas son desconocidas. Se puede ver en un caso que el maximo es 999, lo cuál es un claro indicio de que así se marcan valores numéricos desconocidos. Esto será importante tenerlo en cuenta en siguientes pasos. Similar a como funciona summary, es posible usar describe para estadísticas numéricas más completas sin necesidad de especificarlas.

In [0]:
print("\nVEHÍCULOS - Describe:")
print("Numéricas:")
vehiculos_involucrados.select("edad_pil", "mes_ocu", "hora_ocu").describe().show()
print("\nCategóricas:")
hechos_transito.select("mes_ocu", "hora_ocu", "dia_ocu").describe().show()
print("Numéricas:")
fallecidos_lesionados.select("edad_pil", "mes_ocu", "hora_ocu").describe().show()



VEHÍCULOS - Describe:
Numéricas:
+-------+------------------+-----------------+------------------+
|summary|          edad_pil|          mes_ocu|          hora_ocu|
+-------+------------------+-----------------+------------------+
|  count|            111988|           111988|            111988|
|   mean| 256.2040040004286|6.597055041611601|13.702557416866092|
| stddev|407.41133258885975| 3.51955144121383| 7.020649994962808|
|    min|               3.0|              1.0|               0.0|
|    max|             999.0|             12.0|              99.0|
+-------+------------------+-----------------+------------------+


Categóricas:
+-------+------------------+------------------+------------------+
|summary|           mes_ocu|          hora_ocu|           dia_ocu|
+-------+------------------+------------------+------------------+
|  count|             76759|             76759|             76759|
|   mean|6.5776781875740955|13.719928607720266|15.616123190765904|
| stddev| 3.5268926483

In [0]:
fallecidos_lesionados.printSchema()
hechos_transito.printSchema()
vehiculos_involucrados.printSchema()

root
 |-- num_hecho: double (nullable = true)
 |-- dia_ocu: double (nullable = true)
 |-- mes_ocu: double (nullable = true)
 |-- hora_ocu: double (nullable = true)
 |-- depto_ocu: double (nullable = true)
 |-- mupio_ocu: double (nullable = true)
 |-- areag_ocu: double (nullable = true)
 |-- zona_ocu: double (nullable = true)
 |-- tipo_veh: double (nullable = true)
 |-- causa_acc: double (nullable = true)
 |-- sexo_pil: double (nullable = true)
 |-- edad_pil: double (nullable = true)
 |-- fallecidos_lesionados: double (nullable = true)
 |-- tipo_eve: double (nullable = true)
 |-- archivo_origen: string (nullable = true)
 |-- tipo_dataset: string (nullable = true)
 |-- año: long (nullable = true)

root
 |-- num_hecho: double (nullable = true)
 |-- dia_ocu: double (nullable = true)
 |-- mes_ocu: double (nullable = true)
 |-- dia_sem_ocu: double (nullable = true)
 |-- hora_ocu: double (nullable = true)
 |-- depto_ocu: double (nullable = true)
 |-- mupio_ocu: double (nullable = true)
 |-- a

Con estos resultados nuevamente se ve que hay números grandes y la media y desviación estandar muestran que puede que hayan varios valores atípicos dentro del set de datos.Además de tener en cuenta que varios valores son numéricos, debido a que riginalmente eran solo ID's en varios casos.

# Validación de años

Tomando en cuenta que todos estos datos provienen de distintos archivos es importante verificar que si tengamos correctamente el rango de los años. En el pre procesamiento de datos si alguno no tenía año se lo hemos colocado dependiendo del archivo de origen, por lo que debería de estar el rango completo.

In [0]:
fallecidos_lesionados.select("año").distinct().show()
hechos_transito.select("año").distinct().show()
vehiculos_involucrados.select("año").distinct().show()

+----+
| año|
+----+
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
|2019|
|2020|
|2021|
|2022|
|2023|
+----+

+----+
| año|
+----+
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
|2019|
|2020|
|2021|
|2022|
|2023|
+----+

+----+
| año|
+----+
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
|2019|
|2020|
|2021|
|2022|
|2023|
+----+



Como se puede ver todos cumplen con el ranfo de 2013 hasta 2023.

In [0]:
# mostrar los distintos tipos de accidentes
fallecidos_lesionados.select("tipo_eve").distinct().show()
hechos_transito.select("causa_acc").distinct().show()
vehiculos_involucrados.select("causa_acc").distinct().show()

+--------+
|tipo_eve|
+--------+
|     1.0|
|     3.0|
|     4.0|
|     2.0|
|     5.0|
|    99.0|
|     7.0|
|     8.0|
|     6.0|
|    NULL|
+--------+

+---------+
|causa_acc|
+---------+
|      2.0|
|      1.0|
|      3.0|
|      4.0|
|     99.0|
|      5.0|
|     NULL|
+---------+

+---------+
|causa_acc|
+---------+
|      2.0|
|      1.0|
|      3.0|
|      4.0|
|     99.0|
|     NULL|
+---------+



Estos valores son números porque seguramente dentro de los respectivos archivos eran ID's. 

In [0]:
from pyspark.sql.functions import when, col

# Enfoque: Crear nuevas columnas y luego eliminar las originales

# Para fallecidos_lesionados - tipo_eve
fallecidos_lesionados = fallecidos_lesionados.withColumn(
    "tipo_eve_desc",
    when(col("tipo_eve") == 1, "Colisión")
    .when(col("tipo_eve") == 2, "Choque")
    .when(col("tipo_eve") == 3, "Vuelco")
    .when(col("tipo_eve") == 4, "Caída")
    .when(col("tipo_eve") == 5, "Atropello")
    .when(col("tipo_eve") == 6, "Pérdida de control")
    .when(col("tipo_eve") == 7, "Colisión contra animal")
    .when(col("tipo_eve") == 8, "Exceso de pasaje")
    .when(col("tipo_eve") == 9, "Asfalto mojado")
    .when(col("tipo_eve") == 10, "Exceso de velocidad")
    .when(col("tipo_eve") == 11, "Desperfectos mecánicos")
    .when(col("tipo_eve") == 12, "Incendio")
    .when(col("tipo_eve") == 99, "Ignorado")
    .otherwise("Otro")
)

# Para hechos_transito - causa_acc
hechos_transito = hechos_transito.withColumn(
    "causa_acc_desc",
    when(col("causa_acc") == 1, "Colisión")
    .when(col("causa_acc") == 2, "Choque")
    .when(col("causa_acc") == 3, "Vuelco")
    .when(col("causa_acc") == 4, "Caída")
    .when(col("causa_acc") == 5, "Atropello")
    .when(col("causa_acc") == 6, "Derrape")
    .when(col("causa_acc") == 7, "Embarranco")
    .when(col("causa_acc") == 8, "Encuneto")
    .when(col("causa_acc") == 99, "Ignorado")
    .otherwise("Otro")
)

# Para vehiculos_involucrados - causa_acc
vehiculos_involucrados = vehiculos_involucrados.withColumn(
    "causa_acc_desc",
    when(col("causa_acc") == 1, "Colisión")
    .when(col("causa_acc") == 2, "Choque")
    .when(col("causa_acc") == 3, "Vuelco")
    .when(col("causa_acc") == 4, "Caída")
    .when(col("causa_acc") == 5, "Atropello")
    .when(col("causa_acc") == 6, "Derrape")
    .when(col("causa_acc") == 7, "Embarranco")
    .when(col("causa_acc") == 8, "Encuneto")
    .when(col("causa_acc") == 99, "Ignorado")
    .otherwise("Otro")
)


In [0]:
print("FALLECIDOS_LESIONADOS - Tipos de evento únicos:")
fallecidos_lesionados.select("tipo_eve_desc").distinct().show()

print("HECHOS_TRANSITO - Causas de accidente únicas:")
hechos_transito.select("causa_acc_desc").distinct().show()

print("VEHICULOS_INVOLUCRADOS - Causas de accidente únicas:")
vehiculos_involucrados.select("causa_acc_desc").distinct().show()


FALLECIDOS_LESIONADOS - Tipos de evento únicos:
+--------------------+
|       tipo_eve_desc|
+--------------------+
|                Otro|
|            Colisión|
|              Vuelco|
|               Caída|
|              Choque|
|           Atropello|
|            Ignorado|
|Colisión contra a...|
|    Exceso de pasaje|
|  Pérdida de control|
+--------------------+

HECHOS_TRANSITO - Causas de accidente únicas:
+--------------+
|causa_acc_desc|
+--------------+
|        Choque|
|      Colisión|
|        Vuelco|
|         Caída|
|      Ignorado|
|     Atropello|
|          Otro|
+--------------+

VEHICULOS_INVOLUCRADOS - Causas de accidente únicas:
+--------------+
|causa_acc_desc|
+--------------+
|        Choque|
|      Colisión|
|        Vuelco|
|         Caída|
|      Ignorado|
|          Otro|
+--------------+



Con esto se puede ver los distintos tipos de eventos y accidentes por cada uno de los sets de datos, colocando una columna nueva con el valor del tipo de accidente. Los valores fueron extaidos directamente del diccionario de varaibles del ultimo año usado, 2023.

# Departamentos por set de datos

In [0]:
# Departamentos unicos por set de datos, de igual manera es una columna numerica entonces hay que reemplazarlo por sus valores reales.

from pyspark.sql.functions import when, col

# Aplicar la transformación para departamentos en los tres datasets

# Para fallecidos_lesionados - depto_ocu
fallecidos_lesionados = fallecidos_lesionados.withColumn(
    "depto_ocu_desc",
    when(col("depto_ocu") == 1, "Guatemala")
    .when(col("depto_ocu") == 2, "El progreso")
    .when(col("depto_ocu") == 3, "Sacatepéquez")
    .when(col("depto_ocu") == 4, "Chimaltenango")
    .when(col("depto_ocu") == 5, "Escuintla")
    .when(col("depto_ocu") == 6, "Santa rosa")
    .when(col("depto_ocu") == 7, "Sololá")
    .when(col("depto_ocu") == 8, "Totonicapán")
    .when(col("depto_ocu") == 9, "Quetzaltenango")
    .when(col("depto_ocu") == 10, "Suchitepéquez")
    .when(col("depto_ocu") == 11, "Retalhuleu")
    .when(col("depto_ocu") == 12, "San Marcos")
    .when(col("depto_ocu") == 13, "Huehuetenango")
    .when(col("depto_ocu") == 14, "Quiche")
    .when(col("depto_ocu") == 15, "Baja Verapaz")
    .when(col("depto_ocu") == 16, "Alta Verapaz")
    .when(col("depto_ocu") == 17, "Peten")
    .when(col("depto_ocu") == 18, "Izabal")
    .when(col("depto_ocu") == 19, "Zacapa")
    .when(col("depto_ocu") == 20, "Chiquimula")
    .when(col("depto_ocu") == 21, "Jalapa")
    .when(col("depto_ocu") == 22, "Jutiapa")
    .otherwise("Desconocido")
)

# Para hechos_transito - depto_ocu
hechos_transito = hechos_transito.withColumn(
    "depto_ocu_desc",
    when(col("depto_ocu") == 1, "Guatemala")
    .when(col("depto_ocu") == 2, "El progreso")
    .when(col("depto_ocu") == 3, "Sacatepéquez")
    .when(col("depto_ocu") == 4, "Chimaltenango")
    .when(col("depto_ocu") == 5, "Escuintla")
    .when(col("depto_ocu") == 6, "Santa rosa")
    .when(col("depto_ocu") == 7, "Sololá")
    .when(col("depto_ocu") == 8, "Totonicapán")
    .when(col("depto_ocu") == 9, "Quetzaltenango")
    .when(col("depto_ocu") == 10, "Suchitepéquez")
    .when(col("depto_ocu") == 11, "Retalhuleu")
    .when(col("depto_ocu") == 12, "San Marcos")
    .when(col("depto_ocu") == 13, "Huehuetenango")
    .when(col("depto_ocu") == 14, "Quiche")
    .when(col("depto_ocu") == 15, "Baja Verapaz")
    .when(col("depto_ocu") == 16, "Alta Verapaz")
    .when(col("depto_ocu") == 17, "Peten")
    .when(col("depto_ocu") == 18, "Izabal")
    .when(col("depto_ocu") == 19, "Zacapa")
    .when(col("depto_ocu") == 20, "Chiquimula")
    .when(col("depto_ocu") == 21, "Jalapa")
    .when(col("depto_ocu") == 22, "Jutiapa")
    .otherwise("Desconocido")
)

# Para vehiculos_involucrados - depto_ocu
vehiculos_involucrados = vehiculos_involucrados.withColumn(
    "depto_ocu_desc",
    when(col("depto_ocu") == 1, "Guatemala")
    .when(col("depto_ocu") == 2, "El progreso")
    .when(col("depto_ocu") == 3, "Sacatepéquez")
    .when(col("depto_ocu") == 4, "Chimaltenango")
    .when(col("depto_ocu") == 5, "Escuintla")
    .when(col("depto_ocu") == 6, "Santa rosa")
    .when(col("depto_ocu") == 7, "Sololá")
    .when(col("depto_ocu") == 8, "Totonicapán")
    .when(col("depto_ocu") == 9, "Quetzaltenango")
    .when(col("depto_ocu") == 10, "Suchitepéquez")
    .when(col("depto_ocu") == 11, "Retalhuleu")
    .when(col("depto_ocu") == 12, "San Marcos")
    .when(col("depto_ocu") == 13, "Huehuetenango")
    .when(col("depto_ocu") == 14, "Quiche")
    .when(col("depto_ocu") == 15, "Baja Verapaz")
    .when(col("depto_ocu") == 16, "Alta Verapaz")
    .when(col("depto_ocu") == 17, "Peten")
    .when(col("depto_ocu") == 18, "Izabal")
    .when(col("depto_ocu") == 19, "Zacapa")
    .when(col("depto_ocu") == 20, "Chiquimula")
    .when(col("depto_ocu") == 21, "Jalapa")
    .when(col("depto_ocu") == 22, "Jutiapa")
    .otherwise("Desconocido")
)


In [0]:
# ver lo nuevo
print("FALLECIDOS_LESIONADOS - Departamento:")
fallecidos_lesionados.select("depto_ocu_desc").distinct().show()
print("HECHOS_TRANSITO - Departamento:")
hechos_transito.select("depto_ocu_desc").distinct().show()
print("VEHICULOS_INVOLUCRADOS - Departamento:")
vehiculos_involucrados.select("depto_ocu_desc").distinct().show()

# mostrar el largo de las columnas depto_ocu_desc
print("FALLECIDOS_LESIONADOS - Departamento:")
print(fallecidos_lesionados.select("depto_ocu_desc").distinct().count())

print("HECHOS_TRANSITO - Departamento:")
print(hechos_transito.select("depto_ocu_desc").distinct().count())

print("VEHICULOS_INVOLUCRADOS - Departamento:")
print(vehiculos_involucrados.select("depto_ocu_desc").distinct().count())

FALLECIDOS_LESIONADOS - Departamento:
+--------------+
|depto_ocu_desc|
+--------------+
|     Guatemala|
|        Zacapa|
|     Escuintla|
|    San Marcos|
| Huehuetenango|
|        Izabal|
|  Sacatepéquez|
|  Alta Verapaz|
|    Santa rosa|
|Quetzaltenango|
|   El progreso|
|        Sololá|
|    Chiquimula|
| Suchitepéquez|
|  Baja Verapaz|
|   Totonicapán|
|        Quiche|
| Chimaltenango|
|       Jutiapa|
|         Peten|
+--------------+
only showing top 20 rows
HECHOS_TRANSITO - Departamento:
+--------------+
|depto_ocu_desc|
+--------------+
|     Guatemala|
|        Zacapa|
|     Escuintla|
|Quetzaltenango|
|    San Marcos|
| Huehuetenango|
|  Alta Verapaz|
|        Izabal|
|  Sacatepéquez|
|    Santa rosa|
|   El progreso|
|        Sololá|
|    Chiquimula|
| Suchitepéquez|
|  Baja Verapaz|
|    Retalhuleu|
|   Totonicapán|
|        Quiche|
| Chimaltenango|
|       Jutiapa|
+--------------+
only showing top 20 rows
VEHICULOS_INVOLUCRADOS - Departamento:
+--------------+
|depto_o

Podemos ver que todos los departamentos tienen al menos un evento, accidente o similar rpeortado en cada uno de las tablas. Y hemos hechp una columna depto_occu_desc para poder tener el valor de string y non solo el ID.

In [0]:
from pyspark.sql import functions as F

print("Número de accidentes por año y departamento - FALLECIDOS_LESIONADOS")
df_fallecidos_agrupado = fallecidos_lesionados.groupBy("año", "depto_ocu_desc").count().orderBy("año", "depto_ocu_desc")
display(df_fallecidos_agrupado)

print("Número de accidentes por año y departamento - HECHOS_TRANSITO")
df_hechos_agrupado = hechos_transito.groupBy("año", "depto_ocu_desc").count().orderBy("año", "depto_ocu_desc")
display(df_hechos_agrupado)

print("Número de accidentes por año y departamento - VEHICULOS_INVOLUCRADOS")
df_vehiculos_agrupado = vehiculos_involucrados.groupBy("año", "depto_ocu_desc").count().orderBy("año", "depto_ocu_desc")
display(df_vehiculos_agrupado)

Número de accidentes por año y departamento - FALLECIDOS_LESIONADOS


año,depto_ocu_desc,count
2013,Alta Verapaz,317
2013,Baja Verapaz,191
2013,Chimaltenango,401
2013,Chiquimula,271
2013,El progreso,200
2013,Escuintla,675
2013,Guatemala,2648
2013,Huehuetenango,256
2013,Izabal,310
2013,Jalapa,138


Databricks visualization. Run in Databricks to view.

Número de accidentes por año y departamento - HECHOS_TRANSITO


año,depto_ocu_desc,count
2013,Alta Verapaz,219
2013,Baja Verapaz,100
2013,Chimaltenango,212
2013,Chiquimula,204
2013,El progreso,132
2013,Escuintla,424
2013,Guatemala,2267
2013,Huehuetenango,201
2013,Izabal,234
2013,Jalapa,89


Databricks visualization. Run in Databricks to view.

Número de accidentes por año y departamento - VEHICULOS_INVOLUCRADOS


año,depto_ocu_desc,count
2013,Alta Verapaz,162
2013,Baja Verapaz,109
2013,Chimaltenango,143
2013,Chiquimula,161
2013,El progreso,152
2013,Escuintla,494
2013,Guatemala,2423
2013,Huehuetenango,177
2013,Izabal,230
2013,Jalapa,92


Databricks visualization. Run in Databricks to view.

Analizando las gráficas es evidente que el lugar con mayor cantidad de accentes con diferencia es la capital de Guatemala. Esto tiene sentido considerando que es aquí el lugar donde hay una mayor concentración de presonas y carros.

# Distribución de accidentes por día en 2023

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col

# Verificar los valores reales en la columna
print("Valores únicos en dia_sem_ocu:")
hechos_transito.select("dia_sem_ocu").distinct().orderBy("dia_sem_ocu").show()

# Verificar cuántos registros tienen valores nulos en dia_sem_ocu para 2023
nulos_2023 = hechos_transito.filter(F.col("año") == 2023).filter(F.col("dia_sem_ocu").isNull()).count()
print(f"Registros con dia_sem_ocu nulo en 2023: {nulos_2023}")

# Corregir la transformación - comparar con valores double
hechos_transito = hechos_transito.withColumn(
    "dia_semana_desc",
    F.when(F.col("dia_sem_ocu") == 1.0, "Domingo")
    .when(F.col("dia_sem_ocu") == 2.0, "Lunes")
    .when(F.col("dia_sem_ocu") == 3.0, "Martes")
    .when(F.col("dia_sem_ocu") == 4.0, "Miércoles")
    .when(F.col("dia_sem_ocu") == 5.0, "Jueves")
    .when(F.col("dia_sem_ocu") == 6.0, "Viernes")
    .when(F.col("dia_sem_ocu") == 7.0, "Sábado")
    .otherwise("Desconocido")
)

# Contar accidentes por día de semana en 2023
accidentes_2023_por_dia = hechos_transito.filter(F.col("año") == 2020) \
    .groupBy("dia_semana_desc") \
    .count() \
    .orderBy("count", ascending=False)

print("Accidentes por día de la semana en 2023:")
accidentes_2023_por_dia.show()

print("GRÁFICA DE COLUMNAS - Accidentes por día de la semana en 2023")
display(accidentes_2023_por_dia)


Valores únicos en dia_sem_ocu:
+-----------+
|dia_sem_ocu|
+-----------+
|        1.0|
|        2.0|
|        3.0|
|        4.0|
|        5.0|
|        6.0|
|        7.0|
+-----------+

Registros con dia_sem_ocu nulo en 2023: 0
Muestra de datos transformados:
+-----------+---------------+
|dia_sem_ocu|dia_semana_desc|
+-----------+---------------+
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
|        7.0|         Sábado|
+-----------+---------------+
only showing top 20 

dia_semana_desc,count
Viernes,1180
Sábado,1055
Domingo,880
Jueves,873
Lunes,796
Martes,783
Miércoles,783


Databricks visualization. Run in Databricks to view.

En general se puede ver que los días viernes son los más peligrosos con respecto a la cantidad accidentes en el año 2023.

# Distribución de accidentes por hora en el día, en el municipio Guatemala

In [0]:
from pyspark.sql import functions as F

# solo guate
accidentes_guatemala = hechos_transito.filter(F.col("mupio_ocu") == 101.0)

print("Número de accidentes en municipio de Guatemala:")
accidentes_guatemala.count()

# Crear la columna de hora
distribucion_horas = accidentes_guatemala.withColumn(
    "hora", F.floor(F.col("hora_ocu"))
).filter(
    F.col("hora").between(0, 23)  # Filtrar horas válidas
).groupBy(
    "hora"
).agg(
    F.count("*").alias("cantidad_accidentes")
).orderBy(
    "hora"
)

# Mostrar los resultados
print("Distribución de accidentes por hora en municipio de Guatemala:")

display(distribucion_horas)

Número de accidentes en municipio de Guatemala:
Distribución de accidentes por hora en municipio de Guatemala:


hora,cantidad_accidentes
0,737
1,734
2,575
3,472
4,370
5,417
6,452
7,559
8,512
9,587


Databricks visualization. Run in Databricks to view.

Como se puede ver en la gráfica, es más peligroso en la noche. Parece que ha medida que avanza el día, aumenta la cantidad de accidentes.